## Results for AlexNet Overfit (CIFAR10)

Goal: Regress Validation Accuracy with Train Accuracy optionally with MLH 

|         Method         | R^2 Value | PearsonR |
|:---------------------------:|:---------:|:-------:|
| TrainAcc + MLH              |-1202.2837 |0.5742 |
| (TrainAcc + MLH) / 2        | -37.2667  |0.7818 |
| GPR (TrainAcc Only)         | -0.0205   | 0.1887 | 
| GPR (TrainAcc & MLH)        |  0.8917  | 0.9444 | **bold text**

In [1]:
import sklearn.gaussian_process as gp

In [2]:
# organize dataset
path = "./energy_vs_val_alexnet_none_regression"
import os
import json
X_train = []
Y_train = []
X_test = []
Y_test = []

steps_all = []
benfords_all = []
train_all = []
val_all = []

ap = []

for js in os.listdir(path):
  steps = []
  benfords = []
  train = []
  val = []
  file = os.path.join(path, js)
  dic = json.load(open(file))
  # print(dic.keys())
  # print(len(dic['benford_list']), len(dic['val_acc_step']), len(dic['train_acc_step']))
  for val_dic in dic["val_acc_step"][:-1]:
    val.append(val_dic["validation_accuracy"])
    steps.append(int(val_dic["step"])+1)

  train_sum = []
  ben_sum = []

  for tr_dic in dic["train_acc_step"]:

    train_sum.append(float(tr_dic["training_acc"]))
    ben_sum.append(float(dic["benford_list"][tr_dic["step"]-1]))

    if tr_dic["step"] in steps:
      ap.append(tr_dic["step"])
      train.append(sum(train_sum)/len(train_sum))
      benfords.append(sum(ben_sum)/len(ben_sum))
      train_sum = []
      ben_sum = []
  # print(steps[-1])
  steps_all.extend(steps)
  benfords_all.extend(benfords)
  train_all.extend(train)
  val_all.extend(val)

    

print(len(steps_all), len(benfords_all), len(train_all), len(val_all))


12000 12000 12000 12000


In [ ]:
# val, train

In [3]:
from sklearn.model_selection import train_test_split
X = list(zip(train_all, benfords_all))
Y = val_all
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [4]:
kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0, (1e-3, 1e3))

In [5]:
model = gp.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1, normalize_y=True)

In [7]:
model.fit(X_train, Y_train)
params = model.kernel_.get_params()

In [8]:
y_pred, std = model.predict(X_test, return_std=True)

In [9]:
MSE = ((y_pred-Y_test)**2).mean()
print(MSE)

0.00017124137984718345


In [10]:
print(params)

{'k1': 3.95**2, 'k2': RBF(length_scale=0.0356), 'k1__constant_value': 15.60459890648993, 'k1__constant_value_bounds': (0.1, 1000.0), 'k2__length_scale': 0.03563257340024246, 'k2__length_scale_bounds': (0.001, 1000.0)}


In [11]:
from sklearn.metrics import r2_score
r2_score(Y_test, y_pred)

0.8916696158605524

In [12]:
import numpy as np
a = np.array(train_all)
b = np.array(benfords_all)

Y_pred_sum = a + b

In [13]:
# print(Y_pred_sum[:-10], Y[:-10])

In [14]:
r2_score(Y, Y_pred_sum)

-1202.2836949148962

In [15]:
from scipy.stats import pearsonr
pearsonr(Y, Y_pred_sum)

(0.5741827056437445, 0.0)

In [16]:
pearsonr(Y_test, y_pred)

(0.944442610650453, 0.0)

In [28]:
import numpy as np
from sklearn.metrics import r2_score
a = []
b = []
for tr, ben in X_test:
  a.append(tr)
  b.append(ben)


a = np.array(a)
b = np.array(b)

# a = (a - a.min()) / (a.max() - a.min())
b = (b - b.min()) / (b.max() - b.min())

Y_pred_sum = (0.5*a) + (0.5*b)
r2_score(Y_test, Y_pred_sum)

TypeError: cannot unpack non-iterable float object

In [18]:
pearsonr(Y_test, Y_pred_sum)

(0.7818399731347642, 0.0)

In [ ]:
%cd /content/drive/MyDrive/benfords_law/BenfordsLaw-And-DeepLearning/reg_model

In [ ]:
# !mkdir reg_model
# %cd reg_model/

In [19]:
import pickle
with open("reg_model_overfit", "wb") as f:
  pickle.dump(model, f)

In [ ]:
# val

Without MLH

In [20]:
from sklearn.model_selection import train_test_split
X = train_all
Y = val_all
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [25]:
kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0, (1e-3, 1e3))
model = gp.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1, normalize_y=True)
model.fit(np.array(X_train[:400]).reshape(-1, 1), Y_train[:400])
params = model.kernel_.get_params()
y_pred, std = model.predict(np.array(X_test).reshape(-1, 1), return_std=True)
MSE = ((y_pred-Y_test)**2).mean()
print(MSE)

0.0016132167509015217


/home/surya/miniconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [26]:
r2_score(Y_test, y_pred)

-0.020550000714256944

In [29]:
pearsonr(Y_test, y_pred)

(0.1886590866080153, 4.747692731628452e-33)